In [279]:
import numpy as np
import pandas as pd

In [391]:
age = 26
(age/10)**-2-.0287449295, age-56.3254902
(age/10)**-2*np.log(age/10)-.0510121013

0.09033574559754973

# input

In [280]:
## ----input, echo=FALSE---------------------------------------------------
age_start = 65
screen = 0     # Clinically detected = 0, screen detected = 1
size = 25    # Tumour size mm
grade = 2     # Tumour grade
nodes = 2     # Number positive nodes
er = 1     # ER+ = 1, ER- = 0
her2 = 0     # HER2+ = 1, HER2- = 0, missing = 9
ki67 = 1     # KI67+ = 1, KI67- = 0, missing = 9
pr = 1     # PR+ = 1, PR- = 0, missing = 9

## --- treatment
generation = 2     # Chemo generation 0, 2 or 3 only
horm =  1     # Hormone therapy Yes = 1, no = 0
traz =  0     # Trastuzumab therapy Yes = 1, no = 0
bis =  1     # Bisphosphonate therapy Yes = 1, no = 0
radio =  1     # Radiotherapy Yes = 1, no = 0
heart_gy =  1     # Number of grays radiotherapy to heart

## --- lifestyle
smoker =  1     # Never/ex = 0, current = 1

In [281]:
def ifelse(condition, true_value, false_value):
    return true_value if condition else false_value

def c(start, end):
    return list(range(start, end+1))

def rep(item, num):
    return [item for i in range(num)]

In [282]:
##----------------------------------------------------------------
##[WINTON FIX] Fix inputs for missing data
screen    <- ifelse(screen == 2, 0.204, screen)
grade     <- ifelse(grade == 9, 2.13, grade)
## ------------------------------------------------------------------------
time = c(1,15)
age = map(lambda x: x + age_start - 1, time)
##[WINTON FIX] - Input changed to include grade = 9
grade_val = ifelse(er==1, grade, ifelse(grade>=2, 1, 0)) # Grade variable for ER neg

In [283]:
## ------------------------------------------------------------------------
age_mfp_1 = ifelse(er==1, (age_start/10)**-2-0.0287449295, age_start-56.3254902)
age_beta_1 =  ifelse(er==1, 34.53642, 0.0089827)
age_mfp_2 =  ifelse(er==1, (age_start/10)**-2*np.log(age_start/10)-0.0510121013, 0)
age_beta_2 =  ifelse(er==1, -34.20342, 0)
size_mfp = ifelse(er==1, np.log(size/100)+1.545233938, (size/100)**0.5-.5090456276)
size_beta =  ifelse(er==1, 0.7530729, 2.093446)
nodes_mfp =  ifelse(er==1, np.log((nodes+1)/10)+1.387566896, np.log((nodes+1)/10)+1.086916249)
nodes_beta =  ifelse(er==1, 0.7060723, .6260541)
grade_beta =  ifelse(er==1, 0.746655, 1.129091)
screen_beta = ifelse(er==1, -0.22763366, 0)
her2_beta = ifelse(her2==1, 0.2413,
                      ifelse(her2==0, -0.0762,0 ))
ki67_beta =  ifelse(ki67==1 & er==1, 0.14904,
                      ifelse(ki67==0 & er==1, -0.11333,0 ))
pr_beta = ifelse(pr==1 & er==1, -0.0619,
                      ifelse(pr==0 & er==1, 0.2624,
                             ifelse(pr==1 & er==0, -0.2231,
                                    ifelse(pr==0 & er==0, 0.0296, 0))))

In [284]:
print(age_mfp_1, -0.00507629044674556)
print(age_beta_1,34.53642 )
print(age_mfp_2, -0.00670909119582033)
print(age_beta_2, -34.20342)
print(size_mfp, 0.158939576880109)
print(size_beta, 0.7530729)
print(nodes_mfp, 0.183594091674064)
print(nodes_beta, 0.7060723)
print(grade_beta, 0.746655)
print(screen_beta, -0.22763366)
print(her2_beta, -0.0762)
print(ki67_beta, 0.14904)
print(pr_beta, -0.0619)


-0.005076290446745561 -0.00507629044674556
34.53642 34.53642
-0.006709091195820326 -0.00670909119582033
-34.20342 -34.20342
0.1589395768801094 0.158939576880109
0.7530729 0.7530729
0.18359409167406393 0.183594091674064
0.7060723 0.7060723
0.746655 0.746655
-0.22763366 -0.22763366
-0.0762 -0.0762
0.14904 0.14904
-0.0619 -0.0619


In [285]:
## --- smoking adjustment ------------------------------------------------
smoker_prop = 0.1  # Proportion of population that are current smokers
smoker_rr    = 2    # Relative risk non-br mortality in smokers
cvd_prop     = 0.25 # Proportion of non-br mortality due to smoking related disease
                     # Proportion of I2*, I6*, C15 and C34 in SEARCH breast deaths
smoker_rr_acm = cvd_prop*smoker_rr + 1 - cvd_prop

smoker_beta = ifelse(smoker==0, np.log(1/(1 - smoker_prop + smoker_rr_acm*smoker_prop)),
                      np.log(smoker_rr_acm/(1 - smoker_prop + smoker_rr_acm*smoker_prop))) # Assume smoking RR of 2 and 10% population smokers


In [286]:
print(smoker_beta,0.198450938723838)

0.19845093872383843 0.198450938723838


In [287]:
## ----baseline_adjust-----------------------------------------------------
c_other  = 1.2  # RH non-breast mortality from Kerr et al (2022)
r_prop   = 0.69 # Proportion of population receiving radiotherapy
r_breast = 0.82 # Relative hazard breast specific mortality from Darby et al
r_other  = 1.04 # Relative hazard other mortality per Gy heart dose from Taylor et al (2017)

r_base_br  = np.log(1/((1-r_prop) + r_prop*r_breast))
r_base_oth = np.log(1/((1-r_prop) + r_prop*(r_other**2))) # Assume 2 Gy average heart dose


In [288]:
print(r_base_br,0.132617524614839)
print(r_base_oth,-0.0547760226428012)

0.13261752461483914 0.132617524614839
-0.054776022642801195 -0.0547760226428012


In [289]:
## ------------------------------------------------------------------------
# Other mortality prognostic index (mi)
c_oth = ifelse(generation==0, 0, np.log(c_other))
r_oth = ifelse(radio==0, 0, np.log(r_other)*heart_gy)
mi = 0.0698252*((age_start/10)**2-34.23391957) + r_base_oth + smoker_beta + c_oth + r_oth


In [290]:
print(mi, 0.924941605269109)

0.9249416052691093 0.924941605269109


In [291]:
# Breast cancer mortality prognostic index (pi)
pi = age_beta_1*age_mfp_1 + age_beta_2*age_mfp_2 + size_beta*size_mfp \
    + nodes_beta*nodes_mfp + grade_beta*grade_val +screen_beta*screen \
        + her2_beta + ki67_beta + pr_beta + r_base_br

c     = ifelse(generation == 0, 0, ifelse(generation == 2, -0.248, -0.446))
h     = ifelse(horm==1 & er==1, -0.3857, 0)
h10  = ifelse(h==0, 0, rep(h, 10)+rep(-0.26+h, 5)) #including both ATLAS and aTTom trials
t     = ifelse(her2==1 & traz==1, -.3567, 0)
b     = ifelse(bis==1, -0.198, 0) # Only applicable to menopausal women.
r  = ifelse(radio==1, np.log(r_breast), 0)

rx = pd.DataFrame(rep(pi,15),columns=['s'])
rx['rx'] = r + h + c + t + b + pi
rx['rx10'] = r + h10 + c + t + b + pi

cols = len(rx.columns)


In [292]:
pi,1.94034828035359

(1.9403482803535856, 1.94034828035359)

In [293]:
print(pi)
print(c)
print(h)
print(h10)
print(t)
print(b)
print(r)
print(cols)

1.9403482803535856
-0.248
-0.3857
[-0.3857, -0.3857, -0.3857, -0.3857, -0.3857, -0.3857, -0.3857, -0.3857, -0.3857, -0.3857, -0.6456999999999999, -0.6456999999999999, -0.6456999999999999, -0.6456999999999999, -0.6456999999999999]
0
-0.198
-0.19845093872383832
3


In [326]:
## ------------------------------------------------------------------------
# Generate cumulative baseline other mortality
base_m_cum_oth = np.exp(-6.052919 + (1.079863*np.log(time)) + 0.3255321*np.array(list(map(lambda x: x**0.5, time))))

# Generate cumulative survival non-breast mortality
# Incorporates the increased mortality associated with chemo and radiotherapy
s_cum_oth = np.exp(-np.exp(mi + c_oth + r_oth)*base_m_cum_oth)

# Convert cumulative mortality rate into cumulative risk
m_cum_oth = 1- s_cum_oth

# Annual other mortality rate
m_oth = np.array([i for i in m_cum_oth])

for i in range(1,15):
    m_oth[i] = m_cum_oth[i] - m_cum_oth[i-1]

In [336]:
## ------------------------------------------------------------------------
# Generate cumulative baseline breast mortality
time = np.array(time)
base_m_cum_br = ifelse(
    er==1,
    np.exp(0.7424402 - 7.527762/time**0.5 - 1.812513*np.log(time)/time**0.5),
    np.exp(-1.156036 + 0.4707332/time**2 - 3.51355/time)
    )

In [337]:
# Generate annual baseline breast mortality
base_m_br = np.array([i for i in base_m_cum_br])
for i in range(1,15):
    base_m_br[i] = base_m_cum_br[i] - base_m_cum_br[i-1]


In [382]:
m_br  = pd.DataFrame(base_m_br * np.exp(rx.s), columns=['s'])
m_br['rx'] = base_m_br * np.exp(rx.rx)
m_br['rx10'] = base_m_br * np.exp(rx.rx10)
m_br

,s,rx,rx10
0,0.007868,0.002808,0.002808
1,0.021481,0.007668,0.007668
2,0.030675,0.010950,0.010950
3,0.036552,0.013047,0.013047
4,0.040351,0.014404,0.014404
5,0.042816,0.015283,0.015283
6,0.044391,0.015845,0.015845
7,0.045356,0.016190,0.016190
8,0.045894,0.016382,0.016382
9,0.046127,0.016465,0.016465


In [383]:
rx.rx

0     0.910197
1     0.910197
2     0.910197
3     0.910197
4     0.910197
5     0.910197
6     0.910197
7     0.910197
8     0.910197
9     0.910197
10    0.910197
11    0.910197
12    0.910197
13    0.910197
14    0.910197
Name: rx, dtype: float64

In [384]:
rx.rx10

0     0.910197
1     0.910197
2     0.910197
3     0.910197
4     0.910197
5     0.910197
6     0.910197
7     0.910197
8     0.910197
9     0.910197
10    0.650197
11    0.650197
12    0.650197
13    0.650197
14    0.650197
Name: rx10, dtype: float64